# Running toxicity on The Conversation comments

In [1]:
import pandas as pd
import os
import csv
import json
import sys
import random
import requests
import time
from bs4 import BeautifulSoup

In [2]:
from googleapiclient import discovery
from googleapiclient import http
API_KEY='REPLACE_WITH_YOUR_API_KEY'

In [3]:
service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)
numRequests = 0

def getPrediction(text):
    global numRequests
    global startTime
    
    if len(text) > 20480:
        return -1
    
    analyze_request = {
      'comment': { 'text': text },
      'languages' : ["en"],
      'requestedAttributes': {'TOXICITY': {}}
    }
    
    # 10 requests per second
    
    if numRequests >= 990:
        endTime = time.time()
        while (endTime - startTime) <= 100:
            time.sleep(1)
            endTime = time.time()
        numRequests = 0
        startTime = time.time()

    rateLimitExceeded = True
    while rateLimitExceeded:
        try:
            response = service.comments().analyze(body=analyze_request).execute()
        except http.HttpError:
            time.sleep(100)
        else:
            rateLimitExceeded = False
        finally:
            numRequests += 1

    return response['attributeScores']['TOXICITY']['summaryScore']['value']

## Testing

In [4]:
text = 'I hope your mother gets cervical cancer.'

In [5]:
getPrediction(text)

0.9471205

## Reading in Conversation data

In [6]:
raw = pd.read_csv('/home/vgautam/the_conversation.csv')['Comment body'].apply(BeautifulSoup)

def getText(row):
    return row.text

df = pd.DataFrame(raw.apply(getText).rename('pp_comment_text'))
df = df[df['pp_comment_text'] != '']

/home/vgautam/anaconda3/envs/usra/lib/python3.5/site-packages/pandas/core/series.py:3194: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 3194 of the file /home/vgautam/anaconda3/envs/usra/lib/python3.5/site-packages/pandas/core/series.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  mapped = lib.map_infer(values, f, convert=convert_dtype)


In [7]:
df.head()

,pp_comment_text
0,I agree. For me Persuasion edges out Austen’s ...
1,I’m not sure there is a ready formula for incu...
2,"FISHING EXPEDITIONS?Yes, I also saw the “fishi..."
3,"Prof LaBella, while I totally agree with major..."
4,You are correct. Einstein is an exception. ...


## Predicting toxicity

In [8]:
startTime = time.time()
df['TOXICITY_probability'] = df['pp_comment_text'].apply(getPrediction)

## Preview of results

In [11]:
df.head()

,pp_comment_text,TOXICITY_probability
0,I agree. For me Persuasion edges out Austen’s ...,0.100128
1,I’m not sure there is a ready formula for incu...,0.146383
2,"FISHING EXPEDITIONS?Yes, I also saw the “fishi...",0.069758
3,"Prof LaBella, while I totally agree with major...",0.075774
4,You are correct. Einstein is an exception. ...,0.106335


In [12]:
df.to_csv('/home/vgautam/conversation_toxicity_results.csv', index=False)